In [1]:
from rapper import custom_endpoint

/opt/miniconda3/envs/mitm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-29 08:40:00,982 - Loading faiss.
2024-06-29 08:40:00,991 - Successfully loaded faiss.


In [39]:
from typing import Any, Dict, List, Optional, Union
import json
import requests

from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM
from langchain_core.pydantic_v1 import Field, SecretStr

class SnovaLLM(LLM):
    """Custom LLM for Snova AI API integration with LangChain.

    This LLM uses the Snova AI API to generate responses based on given prompts.

    Attributes:
        model (str): The model to use for generation. Defaults to "llama3-8b-typhoon".
        temperature (float): The sampling temperature. Defaults to 0.7.
        top_k (int): The number of top tokens to consider. Defaults to 50.
        top_p (float): The cumulative probability threshold for top-p sampling. Defaults to 0.95.
        base_url (str): The base URL for the Snova AI API. Defaults to "https://kjddazcq2e2wzvzv.snova.ai/api/v1/chat/completion".
        api_key (SecretStr): The API key for authentication.

    Example:
        .. code-block:: python

            llm = SnovaLLM(api_key="your-api-key-here")
            result = llm.invoke("Tell me a joke about programming.")
    """

    model: str = Field(default="llama3-8b-typhoon")
    temperature: float = Field(default=0.7)
    top_k: int = Field(default=50)
    top_p: float = Field(default=0.95)
    max_tokens_to_generate: int = Field(default=4096)
    base_url: str = Field(default="https://kjddazcq2e2wzvzv.snova.ai/api/v1/chat/completion")
    api_key: SecretStr = Field(...)

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        """Generate a response using the Snova AI API.

        Args:
            prompt (str): The prompt to generate from.
            stop (Optional[List[str]]): Stop sequences for generation.
            run_manager (Optional[CallbackManagerForLLMRun]): Callback manager for the run.
            **kwargs: Additional keyword arguments.

        Returns:
            str: The generated response.
        """
        headers = {
            "Authorization": f"Basic {self.api_key.get_secret_value()}",
            "Content-Type": "application/json"
        }

        data = {
            "inputs": [
                {"role": "system", "content": "You are a helpful AI assistant."},
                {"role": "user", "content": prompt},
            ],
            "max_tokens_to_generate": self.max_tokens_to_generate,
            "temperature": self.temperature,
            "top_k": self.top_k,
            "top_p": self.top_p,
            "model": self.model,
            "stop": stop or ["<|eot_id|>", "<|end_of_text|>"],
        }

        response = requests.post(self.base_url, headers=headers, data=json.dumps(data))
        response.raise_for_status()

        lines_result = response.text.strip().split("\n")
        text_result = lines_result[-1].replace("data: ", '')
        out = json.loads(text_result)

        return out['completion']

    @property
    def _llm_type(self) -> str:
        """Return the type of LLM."""
        return "snova_ai"

    @property
    def _identifying_params(self) -> Dict[str, Any]:
        """Get the identifying parameters."""
        return {
            "model_name": self.model,
            "temperature": self.temperature,
            "top_k": self.top_k,
            "top_p": self.top_p,
        }

In [40]:

llm = SnovaLLM(api_key="")
result = llm.invoke("Tell me a joke about programming.")


In [55]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [("system", "You are a helpful assistant named Typhoon. You always answer in Thai."), ("human", "{input}")]
)
chain = prompt | llm

In [56]:
prompt.invoke("Hwllo")

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant named Typhoon. You always answer in Thai.'), HumanMessage(content='Hwllo')])

In [57]:
chain.invoke("คุณชื่ออะไร")

'ฉันชื่อไต้ฝุ่น (Typhoon)'

# Test Agentic

In [58]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

def get_retriver(url):
    loader = WebBaseLoader(url)
    docs = loader.load()
    documents = RecursiveCharacterTextSplitter(
        chunk_size=1000, chunk_overlap=200
    ).split_documents(docs)
    vector = FAISS.from_documents(documents, OpenAIEmbeddings())
    retriever = vector.as_retriever()
    return retriever

In [59]:
from langchain.tools.retriever import create_retriever_tool

retriever_langsmith_tool = create_retriever_tool(
    get_retriver("https://docs.smith.langchain.com/overview"),
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

retriever_scb10x_tool = create_retriever_tool(
    get_retriver("https://www.blognone.com/node/131737"),
    "scb10x_mission",
    "Search for information about SCB10x. For any questions about SCB10x, you must use this tool!",
)

tools = [retriever_langsmith_tool, retriever_scb10x_tool]

2024-06-29 09:13:55,732 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-06-29 09:13:56,970 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [68]:
tools

[Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x3088b1000>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x309a2f8b0>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x3088b11b0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x309a2f8b0>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n')),
 Tool(name='scb10x_mission', description='Search for information about SCB10x. For any questions ab

In [60]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [73]:
# https://python.langchain.com/v0.1/docs/use_cases/tool_use/prompting/
from operator import itemgetter
from langchain.tools.render import render_text_description
from langchain_core.output_parsers import JsonOutputParser

def tool_chain(model_output):
    tool_map = {tool.name: tool for tool in tools}
    chosen_tool = tool_map[model_output["name"]]
    return itemgetter("arguments") | chosen_tool

rendered_tools = render_text_description(tools)
system_prompt = f"""You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use. Return your response as a JSON blob with 'name' and 'arguments' keys."""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

chain = prompt | llm | JsonOutputParser() | tool_chain
chain.invoke({"input": "What is SCB10x answer in Thai", 'page_content': ''})

2024-06-29 09:23:18,625 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


'ก้าวต่อไปของ SCB 10X กับการปั้นยูนิคอร์นไทย และเป้าหมายเบอร์ 1 CVC\n\n\n\nBy: workplace  on 6 December 2022 - 10:30\nTags:Topics:\xa0SCB10XSCBBlognone Workplace\n\n\n\n\nกลุ่มธนาคารไทยพาณิชย์เป็นหนึ่งในธนาคารแรกๆ ในไทยที่มีการปรับตัวเพื่อเปลี่ยนผ่านสู่สิ่งใหม่ๆ สำหรับด้านอุตสาหกรรมการเงินและการธนาคาร ตั้งแต่นวัตกรรม เทคโนโลยี ไปจนถึงวัฒนธรรมองค์กร โดยการตั้ง SCB 10X ขึ้นมาเพื่อตอบโจทย์ดังกล่าว\nซึ่งทาง SCB 10X เองก็ผ่านการเปลี่ยนแปลงทั้งในเชิงโครงสร้างองค์กรและเป้าหมาย จากจุดเริ่มต้นที่เป็นเพียงหน่วยงานย่อยภายใน SCB  เป้าหมายเพื่อสร้างนวัตกรรมใหม่ๆ ก่อนจะกลายมาเป็นบริษัทแยกของตัวเองในปัจจุบัน โดยมีเป้าหมายเพื่อลงทุนและสร้างธุรกิจสตาร์ทอัพใหม่ ๆ ภายใต้แนวคิด Moonshot Mission หรือการคิดและทำ มองเห็นความต้องการของตลาดก่อนคนอื่น\n\nวัฒนธรรมการทำงาน\nความโดดเด่นของ SCB 10X คือการเป็นลูกผสมระหว่างสตาร์ทอัพและบริษัทใหญ่ ทำให้มีบรรยากาศการทำงานเหมือนสตาร์อัพ แต่มีความมั่นคง ในแง่ของความพร้อมเชิงบุคคลากรที่เชี่ยวชาญด้านเทคโนโลยี ด้านการลงทุน พัฒนานวัตกรรม และเงินลงทุน มีรูปแบบการทำงานที่คล่องต